<a href="https://colab.research.google.com/github/anaguilarar/gee_satellite_data/blob/master/examples/2_Superpixel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall pyproj
!pip install pyproj
!pip install wget
!pip install geehydro
!pip install geopandas
!pip install rasterio

Uninstalling pyproj-3.0.1:
  Would remove:
    /usr/local/bin/pyproj
    /usr/local/lib/python3.7/dist-packages/pyproj-3.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libcurl-9a092e29.so.4.6.0
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libnghttp2-223fd912.so.14.17.1
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libproj-27e7e26d.so.19.2.1
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libsqlite3-69e6c45e.so.0.8.6
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libtiff-4e10f209.so.5.5.0
    /usr/local/lib/python3.7/dist-packages/pyproj.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/pyproj/*
Proceed (y/n)? y
  Successfully uninstalled pyproj-3.0.1
  Using cached https://files.pythonhosted.org/packages/b1/72/d52e9ca81caef056062d71991b0e9b1d16af042245627c5d0e4916a36c4f/pyproj-3.0.1-cp37-cp37m-manylinux2010_x86_64.whl


In [3]:
!git clone https://github.com/anaguilarar/gee_satellite_data.git

Cloning into 'gee_satellite_data'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 389 (delta 226), reused 260 (delta 106), pack-reused 0
Receiving objects: 100% (389/389), 1.47 MiB | 14.89 MiB/s, done.
Resolving deltas: 100% (226/226), done.


In [4]:
import os
os.chdir("gee_satellite_data")

In [5]:
import ee
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ZVTRQVwrfJ8-xMm3mQvgPq2SuWRyDsunonPvVxQM0SM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7CnP47F3b0dvKsQd_NZyiVHR8nuE_Hn7MLLJdWLwLo6Tuh6SM-6uw

Successfully saved authorization token.


In [6]:
import pandas as pd
from scripts import gee_satellite_data
from scripts import gee_functions
from scripts import gis_functions
from scripts import general_functions

## Superpixel: Image Segmentation
In the next section we use the segmetation Simple Non-Iterative Clustering (SNIC) module from GEE (https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic) to create image segments. 
Each of these segments will be treated as a superpixel. 

We will use each of the superpixels as the input features for the radar images smoothing. we are going trough following steps, to get a radar images smoothing using superpixels from optical images:
 * Apply image segmention to optical images
 * Get polygons from superpixels
 * reduce each radar image using the oplygons


### Sentinel 2A: Surface Reflectance level

Before downloading data, the code internally removes those areas covered by shadows and clouds. 

In [7]:
se2_toa = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-08-30", 
                                            "data/col_iba.shp", 
                                            "sentinel2_toa",
                                         cloud_percentage= 80)

As a result, we get a list which contains the images available for quering period. Besides a cover percentage feature is calculted, it reffers to ratio of image clean area respect to our area of interest.

In [8]:
se2_toa.summary

,dates,cover_percentage
0,2016-02-09 00:00:00,95.312521
1,2016-02-19 00:00:00,78.831127
2,2016-03-10 00:00:00,54.574607
3,2016-04-19 00:00:00,88.340379
4,2016-04-29 00:00:00,32.501608
5,2016-05-19 00:00:00,95.054495
6,2016-06-08 00:00:00,98.476376
7,2016-06-18 00:00:00,95.943263
8,2016-06-28 15:31:14,1.684517
9,2016-07-08 00:00:00,93.739455


In [9]:
se2_toa.add_vi_layer("ndvi")
cover_percentage = 81

## filter collection by cover percentage
imgcollection = gee_functions.filtering_bycoverpercentage(se2_toa, cover_percentage)
imgcollection = imgcollection.map(lambda img:
                                  img.unmask(0).clip(se2_toa._ee_sp))
## get image segmentation using SNIC method
snic = gee_functions.gee_snic(imgcollection,se2_toa._ee_sp, bands = ["ndvi"] )
### from image segments to polygons
polygons_fromseg = gee_functions.raster_to_polygons(snic, imgcollection.select(["ndvi"]
                                        ).median().clip(se2_toa._ee_sp), se2_toa._ee_sp, scale = 10)



total images: 13 
total images after cover filter: 7


In [10]:
### in this cell we are going to show the collection median, to visualize the image that will be used for segmentation

truecolorParams = {'gamma': 1.3, 
                   'min': 0,
                   'max': 0.3,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = imgcollection.median()
gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, se2_toa.geometry, zoom = 11)

### Sentinel 1 
The instrument onboard the Sentinel-1 satellites transmits in one polarisation (V) and measures in two (V and H), giving the VV and VH bands. The first band in each data file is the signal measured in the same polarisation as the transmission (VV) and the second is measured in the orthogonal polarisation (VH).


In [11]:
s1_imagery = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-08-31", 
                                            "data/col_iba.shp", 
                                            "sentinel1", bands = ['VV'])
s1_imagery.summary

,dates,cover_percentage
0,2016-03-26 23:13:23,99.584903
1,2016-05-13 23:13:30,99.586662
2,2016-02-07 23:13:28,99.551558
3,2016-02-14 10:42:34,99.560758
4,2016-03-02 23:13:28,99.573275
5,2016-03-09 10:42:33,99.572886
6,2016-04-02 10:42:34,99.536986
7,2016-04-19 23:13:29,99.525030
8,2016-04-26 10:42:35,99.534075
9,2016-05-20 10:42:37,99.550265


Due to Sentinel 1 mission has two satellites (a and b), it can orbit the earth in a different orientation (Ascending and Descending), the  6 days data can be summarised averaging over a time window. 

In [12]:
s1_imagery.reduce_collection_by_days(10)
s1_imagery._dates_reduced

0    2016-02-11 04:58:01
1    2016-03-02 23:13:28
2    2016-03-09 10:42:33
3    2016-03-26 23:13:23
4    2016-04-02 10:42:34
5    2016-04-23 04:58:02
6    2016-05-13 23:13:30
7    2016-05-20 10:42:37
8    2016-06-10 04:58:05
9    2016-06-30 23:13:33
10   2016-07-07 10:42:40
11   2016-07-24 23:13:34
12   2016-07-31 10:42:41
dtype: datetime64[ns]

In [13]:
s1datasuperpixel= s1_imagery.image_collection.map(lambda image:
                                gee_functions.reduce_image_to_superpixel(image.select(['VV']).clip(s1_imagery._ee_sp), polygons_fromseg))


Once the superpixels has been obtained, we are going to summary our s1 collection in percentiles

In [14]:
abundance_perc = s1datasuperpixel.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))
abundance_perc_orig = s1_imagery.image_collection.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))



Finally we compare both results

In [15]:
import folium 
display = ee.Image(0).updateMask(0).paint(polygons_fromseg, '000000', 3);

centergeometry = gis_functions.geometry_center(s1_imagery.geometry)

Map = folium.Map(location=[centergeometry[1],
                               centergeometry[0]], zoom_start=12)

Map.addLayer(ee.Image(abundance_perc_orig).select(['VV_p5', 'VV_p50', 'VV_p95']).clip(s1_imagery._ee_sp), {'gamma': 1.3, 'min':-25, 'max':0}, 'orig');
Map.addLayer(ee.Image(abundance_perc).select(['first_p5', 'first_p50', 'first_p95']), {'gamma': 1.3, 'min':-25, 'max':0}, 'radar superpixel');
Map.addLayer(display, {}, 'vectors');


Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

In [ ]:
## download radar images

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Download images
import time
image_todownload = ee.Image(abundance_perc)
band_names = ee.Image(abundance_perc).bandNames().getInfo()
task = ee.batch.Export.image.toDrive(**{
                'image': ee.Image(image_todownload),
                'description': "s1_percentiles_superpixel",
                'folder': '/content/drive/MyDrive/satellite_images',
                'scale': 10,
                'region': se2_toa._ee_sp['coordinates']
            })

task.start()
while task.active():
    print('Polling for task (id: {}).'.format("radar_percentiles_superpixel"))
    time.sleep(20)

In [ ]:
### S2 Metrics
maxvalues = imgcollection.select('ndvi').max()
minvalues = imgcollection.select('ndvi').min()
medianvaluesrgb = imgcollection.select(['B4','B3','B2']).median()
stdvalues = imgcollection.select('ndvi').reduce(ee.Reducer.stdDev())
medianvalues = medianvalues.addBands(maxvalues).addBands(minvalues).addBands(stdvalues)



In [16]:
### spliting the geometry into tiles for processing huge areas

stateArea = se2_toa._ee_sp.area()
stateAreaSqKm = ee.Number(stateArea).divide(1e6).round()
print(stateAreaSqKm.getInfo())

stateArea = s1_imagery._ee_sp.area()
stateAreaSqKm = ee.Number(stateArea).divide(1e6).round()
print(stateAreaSqKm.getInfo())

252
252


### Image segmentation for multiple tiles

In [17]:
###

se2_toa = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-08-30", 
                                            "data/tol_south.shp", 
                                            "sentinel2_toa",
                                         cloud_percentage= 80)
se2_toa.add_vi_layer("ndvi")


cover_percentage = 81

## filter collection by cover percentage
imgcollection = gee_functions.filtering_bycoverpercentage(se2_toa, cover_percentage)
imgcollection = imgcollection.map(lambda img:
                                  img.unmask(0).clip(se2_toa._ee_sp))


total images: 13 
total images after cover filter: 7


In [18]:
s1_imagery = gee_satellite_data.get_gee_data("2016-02-01",
                                            "2016-08-31", 
                                            "data/tol_south.shp", 
                                            "sentinel1", bands = ['VV'])

s1_imagery.reduce_collection_by_days(10)
s1_imagery._dates_reduced

0    2016-02-12 06:53:00
1    2016-03-02 23:13:28
2    2016-03-09 10:42:46
3    2016-03-23 11:17:24
4    2016-04-02 10:42:47
5    2016-04-12 23:21:25
6    2016-04-24 06:53:01
7    2016-05-13 23:13:30
8    2016-05-20 10:42:50
9    2016-05-30 23:21:27
10   2016-06-11 11:00:13
11   2016-06-30 23:13:33
12   2016-07-07 10:42:52
13   2016-07-21 11:17:32
14   2016-07-31 10:42:54
15   2016-08-10 23:21:31
dtype: datetime64[ns]

In [21]:
#Get tiles

num_cells = 5

grid_geometries = gis_functions.split_into_tiles(se2_toa._ee_sp['coordinates'][0], num_cells)
eegrid_geometries = [ee.Feature(ee.Geometry.Rectangle(
    grid_geometries[i][0], 
    grid_geometries[i][1], 
    grid_geometries[i][2], grid_geometries[i][3]), 
                              {'label': i}) for i in range(len(grid_geometries))]


import folium
Map = folium.Map(location=[4.247676101204724,-75.22938730661772
                               ], zoom_start=10)

for i in range(len(eegrid_geometries)):
  grid = ee.FeatureCollection(eegrid_geometries[i])
  displaytest = ee.Image(0).updateMask(0).paint(grid, '000000', 3)
  Map.addLayer(displaytest, {}, 'vectors'+str(i));

displaytest = ee.Image(0).updateMask(0).paint(se2_toa._ee_sp, '000000', 3)
Map.addLayer(displaytest, {}, 'original')

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

the segmentation funcition will be applied for each tile

In [23]:
polygons_pertile = []
snic_list = []
superpixels_summary = []
superpixels_quantile = []

for i in range(len(eegrid_geometries)):
  
  tilecollection = imgcollection.map(lambda img:
                                  img.unmask(0).clip(eegrid_geometries[i]))
  ## get image segmentation using SNIC method
  snic = gee_functions.gee_snic(tilecollection,eegrid_geometries[i], bands = ["ndvi"] )
  ### from image segments to polygons
  polygons_fromseg = gee_functions.raster_to_polygons(snic, tilecollection.select(["ndvi"]
                                        ).median(), 
                                            eegrid_geometries[i].geometry(), scale = 10)
  
  ### store outputs in lists
  snic_list.append(snic)
  grid_geometries.append(polygons_fromseg)

  coll_segmentation = s1_imagery.image_collection.map(lambda image:
                                  gee_functions.reduce_image_to_superpixel(image.select(['VV']).clip(eegrid_geometries[i]), 
                                                                           polygons_fromseg))
  
  #superpixels_summary.append(coll_segmentation)
  superpixels_quantile.append(coll_segmentation.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95])))
  

In [ ]:

img_merged = ee.ImageCollection(superpixels_quantile).reduce(ee.Reducer.mean())


In [24]:
### Visualize one tile

import folium 


centergeometry = gis_functions.geometry_center(s1_imagery.geometry)

Map = folium.Map(location=[centergeometry[1],
                               centergeometry[0]], zoom_start=12)

#Map.addLayer(ee.Image(abundance_perc_orig).select(['VV_p5', 'VV_p50', 'VV_p95']).clip(s1_imagery._ee_sp), {'gamma': 1.3, 'min':-25, 'max':0}, 'orig');
Map.addLayer(ee.Image(superpixels_quantile[1]).select(['first_p5', 'first_p50', 'first_p95']), {'gamma': 1.3, 'min':-25, 'max':0}, 'radar superpixel');



Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map

In [ ]:
### Download images
import time

for i in range(len(superpixels_quantile)):
  
  image_todownload = ee.Image(superpixels_quantile[i])

  task = ee.batch.Export.image.toDrive(**{
                'image': image_todownload,
                'description': "s1_percentiles_superpixel_tile_{}_{}".format(i+1, len(superpixels_quantile)),
                'folder': '/content/drive/MyDrive/satellite_images',
                'scale': 10,
                'region': image_todownload.geometry()
            })

  task.start()
  while task.active():
    print('Polling for task (id: {}).'.format("s1_percentiles_superpixel_tile_{}_{}".format(i+1, len(superpixels_quantile))))
    time.sleep(20)

Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_superpixel_tile_1_25).
Polling for task (id: s1_percentiles_sup

In [ ]:
for j in collectionsize
first_image = []
for i in range(len(superpixels_summary)):
  first_image.append(ee.Image(superpixels_summary[i].first()))


imgmerged = ee.ImageCollection(first_image).reduce(ee.Reducer.mean())